# KillSwitch Project: Design Candidate Guide RNAs

## Choose an essential gene

Used Yeastmine to search for genes whose "Gene Phenotype Summary" said they were an "Essential gene". Query code in python below:

In [2]:
from IPython.display import HTML
from intermine.webservice import Service
service = Service("http://yeastmine.yeastgenome.org/yeastmine/service")
query = service.new_query("Gene")
query.add_view(
    "primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol",
    "name", "phenotypeSummary"
)
query.add_constraint("phenotypes.strainBackground", "=", "W303", code = "A")
query.add_constraint("phenotypeSummary", "LIKE", "Essential gene*", code = "B")

columns = ["primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol", "name", "phenotypeSummary"]

output = '<html><table>'
output += '<tr>' + ' '.join(['<th>'+k+'</th>' for k in columns]) + '</tr>'
for row in query.rows():
    output += '<tr>' + ' '.join(['<td>'+str(row[c])+'</td>' for c in columns]) + '</tr>'
output += '</table></html>'
HTML(output)

primaryIdentifier,secondaryIdentifier,organism.shortName,symbol,name,phenotypeSummary
S000000973,YER171W,S. cerevisiae,RAD3,RADiation sensitive,"Essential gene; point mutations result in increased mitotic recombination, elevated mutation frequence, and increased UV sensititvity"
S000002284,YDL126C,S. cerevisiae,CDC48,Cell Division Cycle,"Essential gene; conditional mutations cause cell cycle arrest, abnormal cell morphology, aggregation of mitochondria, elongated spindles that fail to disassemble, sensitivity to cell wall-affecting drugs and oxidative stress; overexpression leads to slow growth"
S000001855,YFL039C,S. cerevisiae,ACT1,ACTin,"Essential gene; conditional mutants show pleiotropic defects including abnormal actin cytoskeleton, chromosome instability, altered subcellular morphology, increased sensitivity to heat, multiple chemicals and osmotic stress; overexpression leads to growth arrest"
S000004550,YML085C,S. cerevisiae,TUB1,TUBulin,"Essential gene; heterozygous diploid null mutants are haploinsufficient and sensitive to heat, TOR inhibitor rapamycin, and actin polymerization inhibitor latrunculin A; conditional mutants have abnormal spindles at cold temperatures; overexpression interferes with the cell cycle and slows growth"
S000003008,YGL040C,S. cerevisiae,HEM2,HEMe biosynthesis,"Essential gene on standard rich medium due to heme auxotrophy; mutants lack ergosterol; in large-scale studies, repressing expression confers increased competitive fitness; heterozygous null diploid shows haploinsufficiency"
S000005785,YOR259C,S. cerevisiae,RPT4,"Regulatory Particle Triple-A protein, or Regulatory Particle Triphosphatase","Essential gene; mutations cause resistance to tunicamycin, sensitivity to heat, cycloheximide, and cadmium ions, slow growth, and accumulation of proteasome substrates; in large-scale studies, repression causes abnormal mitochondrial morphology and an Opi- phenotype (overproduction and excretion of inositol in the absence of inositol and choline); diploid heterozygous null mutant is haploinsufficient and displays enlarged cell size and sensitivity to trapoxin A and 3-nitroso-2-phenylimidazo[1,2-a]pyridine"
S000005643,YOR117W,S. cerevisiae,RPT5,"Regulatory Particle Triple-A protein, or Regulatory Particle Triphosphatase","Essential gene; mutations in the ATPase active site cause sensitivity to heat and canavanine, shorter replicative lifespan, slow growth, and increased budding index; in large-scale studies, repression causes reduced competitive fitness, sensitivity to MMS, resistance to myriocin, and shorter telomere length; diploid heterozygous null mutant exhibits enlarged cell size and haploinsufficiency"
S000001665,YKL182W,S. cerevisiae,FAS1,Fatty Acid Synthetase,Essential gene; null mutant is rescuable by fatty acid supplement; heterozygous diploid accumulates very long-chain fatty acids
S000005046,YNL102W,S. cerevisiae,POL1,POLymerase,"Essential gene; heat-sensitive mutant arrests as large-budded cells with nuclei at the mother-bud neck; point mutations affect minichromosome maintenance, chromosome stability, lifespan, mutation frequency, and telomere length; repression in a diploid causes increases in mitotic and meiotic recombination"
S000005770,YOR244W,S. cerevisiae,ESA1,None,"Essential gene; conditional mutants exhibit heat sensitivity; in a large-scale study, depletion confers an Opi- phenotype, overproduction and excretion of inositol in the absence of inositol and choline"


Out of those results, I selected the following 2 genes:

- **[LCB1 / YMR296C](http://www.yeastgenome.org/locus/S000004911/overview):** This is responsible for sphingolopid synthesis, defects in this gene result in inviablity. I picked it because it said it causes "shmoo formation" which sounded cool.
- **[RPT5 / YOR117W](http://www.yeastgenome.org/locus/S000005643/overview):** This is one of the 6 ATPases, mutations cause short lifespan, shorter telomeres.

## Find candidate PAM sites & design guide RNA

In [ ]:
import coral
from collections import defaultdict

In [ ]:
sites = defaultdict(defaultdict)

In [ ]:
lcb1_promoter = coral.database.get_yeast_promoter_ypa('LCB1')
lcb1_promoter.name = 'LCB1-Promoter'
lcb1_promoter.features.append(coral.Feature('LCB1-Promoter', 0, len(lcb1_promoter), 'promoter'))
lcb1_promoter.display()

In [ ]:
lcb1_cds = coral.database.fetch_yeast_locus_sequence('LCB1')
lcb1_cds.name = 'LCB1'
lcb1_cds.features.append(coral.Feature('LCB1', 0, len(lcb1_cds), 'CDS'))
lcb1_cds.display()

In [ ]:
# find 'NGG' sites, list the last 10 in the promoter (forward direction)
lcb1_promoter.locate('GG')[0][-10:]

In [ ]:
# design guide RNA (20bp 5' from PAM site)
site1 = 983
lcb1_site1 = site1
lcb1_promoter.features.append(coral.Feature('LCB1-pam1', site1-20, 23, 'site'))
lcb1_g1 = lcb1_promoter[site1-20:site1+3]
lcb1_g1.name = 'LCB1-guide1'
lcb1_g1.features.append(coral.Feature('LCB1-guide1', 0, 23, 'misc_RNA'))
lcb1_g1

In [ ]:
site2 = 953-1
lcb1_site2 = site2
lcb1_promoter.features.append(coral.Feature('LCB1-pam2', site2-20, 23, 'site'))
lcb1_g2 = lcb1_promoter[site2-20:site2+3]
lcb1_g2.name = 'LCB1-guide1'
lcb1_g2.features.append(coral.Feature('LCB1-guide2', 0, 23, 'misc_RNA'))
lcb1_g2

In [ ]:
template = lcb1_promoter + lcb1_cds
template[site2-50:].display()

## Make mockup plasmids for each guide RNA

In [ ]:
# import template plasmid
orig_plasmid = coral.seqio.read_dna('./pmod8-pgalz4-urgr.gb')
orig_plasmid.display()

In [ ]:
target = orig_plasmid.select_features('Target Sequence')[0]
print 'len:', target.stop - target.start
orig_plasmid[target.start:target.stop]

In [ ]:
def feat(dna, name):
    return dna.select_features(name)[0]

In [ ]:
def zoom(plasmid):
    zoom = (feat(plasmid,'ASBV1').start, feat(plasmid,'gRNA Handle').stop)
    pz = plasmid[zoom[0]:zoom[1]]
    pz.name += ' (zoom ASBV1..gRNA)'
    return pz.display()

In [ ]:
zoom(orig_plasmid)

In [ ]:
backbone = orig_plasmid.rotate(target.start)[target.stop-target.start:]
backbone.display()

In [ ]:
lcb1_g1_mockup = (lcb1_g1+backbone).circularize()
lcb1_g1_mockup.name = 'pMOD6-killswitch-ASBV1-g1'
zoom(lcb1_g1_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

In [ ]:
lcb1_g2_mockup = (lcb1_g2+backbone).circularize()
lcb1_g2_mockup.name = 'pMOD6-killswitch-ASBV1-g2'
zoom(lcb1_g2_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

## Design primers

In [ ]:
# LCB1 gene, guide for PAM site #1:
lcb1_g1

Hold the annealling region constant (our guide sequence), and design the overhang to bind with the backbone at 65°C.

In [ ]:
p = coral.design.primer(backbone.flip(), tm=65)
lcb1_g1_fwd = coral.Primer(overhang=p.primer().reverse_complement(), anneal=lcb1_g1, tm=p.tm)
lcb1_g1_fwd

In [ ]:
p = coral.design.primer(backbone, tm=65)
lcb1_g1_rev = coral.Primer(overhang=p.primer().reverse_complement(), anneal=lcb1_g1.flip(), tm=p.tm)
lcb1_g1_rev

## Simulate PCR with the primers
Should get back the guide region plus homology with the plasmid on both sides.

In [ ]:
lcb1_g1_pcr = coral.reaction.pcr(lcb1_g1_fwd.primer().to_ds(), lcb1_g1_fwd, lcb1_g1_rev)
lcb1_g1_pcr

In [ ]:
# see if we can find the guide sequence in the output
lcb1_g1_pcr.locate(lcb1_g1)

In [ ]:
# http://54.68.9.194:81/samples/3351
frag1_primers = (
    coral.Primer(anneal=coral.DNA('gttttagagctagaaatagcaagttaaaataaggctagtccg'), tm=71.4),
    coral.Primer(anneal=coral.DNA('ATCGGAGGACCGAAGGAGCTAACC'), tm=72)
)
frag1 = coral.reaction.pcr(orig_plasmid, *frag1_primers)
print len(frag1)
frag1.display()

In [ ]:
# http://54.68.9.194:81/samples/5540
frag2_primers = (
    coral.Primer(anneal=coral.DNA('GGGAGATGGGACGGGCG'), tm=71.86),
    coral.Primer(anneal=coral.DNA('GTTCGCCAGTTAATAGTTTGCGCAACG'), tm=72)
)
frag2 = coral.reaction.pcr(orig_plasmid, *frag2_primers)
print len(frag2)
frag2.display()

In [ ]:
lcb1_g1_gibson = coral.reaction.gibson([frag1, frag2, lcb1_g1_pcr])
lcb1_g1_gibson.display()

In [ ]:
if lcb1_g1_gibson.is_rotation(lcb1_g1_mockup):
    print 'Validated.'

### Repeat for LCB1 site #2

In [ ]:
p = coral.design.primer(backbone.flip(), tm=65)
lcb1_g2_fwd = coral.Primer(overhang=p.primer().reverse_complement(), anneal=lcb1_g2, tm=p.tm)

p = coral.design.primer(backbone, tm=65)
lcb1_g2_rev = coral.Primer(overhang=p.primer().reverse_complement(), anneal=lcb1_g2.flip(), tm=p.tm)

lcb1_g2_pcr = coral.reaction.pcr(lcb1_g2_fwd.primer().to_ds(), lcb1_g2_fwd, lcb1_g2_rev)

lcb1_g2_gibson = coral.reaction.gibson([frag1, frag2, lcb1_g2_pcr])

if lcb1_g2_gibson.is_rotation(lcb1_g2_mockup):
    print 'Validated.'

## Repeat for second gene: RPT5

Download sequence & find candidate PAM sites.

In [ ]:
rpt5_promoter = coral.database.get_yeast_promoter_ypa('RPT5')
print len(rpt5_promoter)
rpt5_promoter.name = 'rpt5-Promoter'
rpt5_promoter.features.append(coral.Feature('rpt5-Promoter', 0, len(rpt5_promoter), 'promoter'))

rpt5_cds = coral.database.fetch_yeast_locus_sequence('rpt5')
rpt5_cds.name = 'RPT5'
rpt5_cds.features.append(coral.Feature('RPT5', 0, len(rpt5_cds), 'CDS'))

# find 'NGG' sites, list the last 10 in the promoter (forward direction)
all_sites = rpt5_promoter.locate('GG')
sites = all_sites[0][-10:]
sites

In [ ]:
site1 = sites[-1]-1
rpt5_promoter[site1-20:site1+3]

In [ ]:
rpt5_site1 = site1
rpt5_promoter.features.append(coral.Feature('rpt5-pam1', site1-20, 23, 'site'))
rpt5_g1 = rpt5_promoter[site1-20:site1+3]
rpt5_g1.name = 'rpt5-guide1'
rpt5_g1.features.append(coral.Feature('rpt5-guide1', 0, 23, 'misc_RNA'))
rpt5_g1

In [ ]:
# let's do the 3rd site rather than 2nd because the 2nd site overlaps with the first guide
site2 = sites[-3]-1
rpt5_site2 = site2
rpt5_promoter.features.append(coral.Feature('rpt5-pam2', site2-20, 23, 'site'))
rpt5_g2 = rpt5_promoter[site2-20:site2+3]
rpt5_g2.name = 'rpt5-guide1'
rpt5_g2.features.append(coral.Feature('rpt5-guide2', 0, 23, 'misc_RNA'))
rpt5_g2

In [ ]:
# make mockups
rpt5_g1_mockup = (rpt5_g1+backbone).circularize()
rpt5_g1_mockup.name = 'pMOD6-killswitch-ASBV1-g1'
#zoom(rpt5_g1_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

rpt5_g2_mockup = (rpt5_g2+backbone).circularize()
rpt5_g2_mockup.name = 'pMOD6-killswitch-ASBV1-g2'
#zoom(rpt5_g2_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

# design primers
p = coral.design.primer(backbone.flip(), tm=65)
rpt5_g1_fwd = coral.Primer(overhang=p.primer().reverse_complement(), anneal=rpt5_g1, tm=p.tm)

p = coral.design.primer(backbone, tm=65)
rpt5_g1_rev = coral.Primer(overhang=p.primer().reverse_complement(), anneal=rpt5_g1.flip(), tm=p.tm)

p = coral.design.primer(backbone.flip(), tm=65)
rpt5_g2_fwd = coral.Primer(overhang=p.primer().reverse_complement(), anneal=rpt5_g2, tm=p.tm)

p = coral.design.primer(backbone, tm=65)
rpt5_g2_rev = coral.Primer(overhang=p.primer().reverse_complement(), anneal=rpt5_g2.flip(), tm=p.tm)


# validate against mockups
rpt5_g1_pcr = coral.reaction.pcr(rpt5_g1_fwd.primer().to_ds(), rpt5_g1_fwd, rpt5_g1_rev)
rpt5_g1_gibson = coral.reaction.gibson([frag1, frag2, rpt5_g1_pcr])

if rpt5_g1_gibson.is_rotation(rpt5_g1_mockup):
    print 'Validated rpt5_g1.'
    
rpt5_g2_pcr = coral.reaction.pcr(rpt5_g2_fwd.primer().to_ds(), rpt5_g2_fwd, rpt5_g2_rev)
rpt5_g2_gibson = coral.reaction.gibson([frag1, frag2, rpt5_g2_pcr])

if rpt5_g2_gibson.is_rotation(rpt5_g2_mockup):
    print 'Validated rpt5_g2.'
